### 1. Setting up spark environment

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

storage_account="mystoacc1kad"
application_id="d3637ee5-7e31-4579-87ed-e334690b695e"
directory_id="26af9d76-35fe-404a-b312-869c37aec9c7"
silver_container_path=f'abfss://silver@{storage_account}.dfs.core.windows.net/Sales/'
gold_container_path=f'abfss://gold@{storage_account}.dfs.core.windows.net/Sales/'

### 2. configuring storage account

In [0]:
service_credential = dbutils.secrets.get(scope="databricks-secrets-sql-migration", key="appsecretvalue")

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
               f"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
               f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

### Function to create and return dataframes from delta files in silver container

In [0]:
def creating_dataframe(df, file_path):
  print(f"Creating dataframes for {df}")
  df=spark.read \
    .format('delta') \
      .option("header", True) \
          .load(f"{silver_container_path}{file_path}")

  return df


### Function to load Delta files into Gold Layer Container

In [0]:
def write_to_silver(df,path):
  df.coalesce(1).write \
    .format('delta') \
      .mode('overwrite') \
        .save(f"{gold_container_path}{path}")
  
  print(f"File saved succesfully ")


## Creating, Transforming & Loading Dataframes to Gold Layer

### 1)   Sales.CountryRegionCurrency 

In [0]:
df_CountryRegionCurrency=creating_dataframe("CountryRegionCurrency", "CountryRegionCurrency/")

In [0]:
write_to_silver(df_CountryRegionCurrency,"CountryRegionCurrency/")

### 2) Sales.CreditCard

In [0]:
df_CreditCard=creating_dataframe("CreditCard", "CreditCard/")

In [0]:
write_to_silver(df_CreditCard,"CreditCard/")

### 3) CurrencyRate

In [0]:
df_CurrencyRate=creating_dataframe("CurrencyRate", "CurrencyRate/")

In [0]:
write_to_silver(df_CurrencyRate,"CurrencyRate/")

### 4) Customer

In [0]:
df_Customer=creating_dataframe("Customer", "Customer/")

- Filling Null Values to zero

In [0]:
df_Customer=df_Customer.fillna({'PersonID': 0, 'storeID': 0})

In [0]:
write_to_silver(df_Customer,"df_Customer/")

### 5) PersonCreditCard

In [0]:
df_PersonCreditCard=creating_dataframe("PersonCreditCard", "PersonCreditCard/")


In [0]:
df_PersonCreditCard.display()

In [0]:
write_to_silver(df_PersonCreditCard,"PersonCreditCard/")

### 6) SalesOrderDetail

In [0]:
df_SalesOrderDetail=creating_dataframe("SalesOrderDetail", "SalesOrderDetail/")

In [0]:
#applying round functionbs on price columns

df_SalesOrderDetail=df_SalesOrderDetail \
    .withColumn("UnitPrice", round(col('UnitPrice'),4)) \
        .withColumn("LineTotal", round(col('LineTotal'),4)) 

In [0]:
df_SalesOrderDetail=df_SalesOrderDetail \
    .withColumn("GrossUnitPrice", col('UnitPrice')*col('OrderQty'))

In [0]:
df_SalesOrderDetail=df_SalesOrderDetail.withColumn("TotalDiscountAmount",  \
    when(col('UnitPriceDiscount') != 0,  round((col('UnitPrice')*col('OrderQty'))*col('UnitPriceDiscount'),4)).otherwise(0) ) 

In [0]:
write_to_silver(df_SalesOrderDetail, 'SalesOrderDetail/')

### 7) SalesOrderHeader

In [0]:
df_SalesOrderHeader=creating_dataframe("SalesOrderHeader", "SalesOrderHeader/")

In [0]:
df_SalesOrderHeader=df_SalesOrderHeader.fillna({'Comment':'NA'})

In [0]:
df_SalesOrderHeader=df_SalesOrderHeader.withColumn("TotalDue", round(col("TotalDue"),2))

### 8) SalesOrderHeaderSalesReason

In [0]:
df_SalesOrderHeaderSalesReason=creating_dataframe("SalesOrderHeaderSalesReason", "SalesOrderHeaderSalesReason/")

In [0]:
write_to_silver(df_SalesOrderHeaderSalesReason, "SalesOrderHeaderSalesReason/")

### 9) SalesPerson

In [0]:
df_SalesPerson=creating_dataframe("SalesPerson", "SalesPerson/")

In [0]:
df_SalesPerson=df_SalesPerson.fillna({'salesQuota': 0 })

In [0]:
write_to_silver(df_SalesPerson,'SalesPerson/')

### 10) SalesPersonQuotaHistory

In [0]:
df_SalesPersonQuotaHistory=creating_dataframe("SalesPersonQuotaHistory", "SalesPersonQuotaHistory/")

In [0]:
write_to_silver(df_SalesPersonQuotaHistory, 'SalesPersonQuotaHistory/')

### 11) SalesReason

In [0]:
df_SalesReason=creating_dataframe("SalesReason", "SalesReason/")

In [0]:
write_to_silver(df_SalesReason,"SalesReason/")

### 12) SalesTaxRate

In [0]:
df_SalesTaxRate=creating_dataframe("SalesTaxRate", "SalesTaxRate/")


In [0]:
write_to_silver(df_SalesReason,'SalesReason/')

### 13) SalesTerritory

In [0]:
df_SalesTerritory=creating_dataframe("SalesTerritory", "SalesTerritory/")

In [0]:
write_to_silver(df_SalesTerritory,'SalesTerritory/')

### 14) SalesTerritoryHistory

In [0]:
df_SalesTerritoryHistory=creating_dataframe("SalesTerritoryHistory", "SalesTerritoryHistory/")

In [0]:
#filling NULL values in EndDate with high end date

df_SalesTerritoryHistory=df_SalesTerritoryHistory \
    .withColumn("EndDate", when(col('EndDate').isNull(), to_date(lit('9999-12-31'))).otherwise(col('EndDate'))) 

In [0]:
write_to_silver(df_SalesTerritoryHistory,"SalesTerritoryHistory/")

### 15) ShoppingCartItem

In [0]:
df_ShoppingCartItem=creating_dataframe("ShoppingCartItem", "ShoppingCartItem/")

### 16) SpecialOffer

In [0]:
df_SpecialOffer=creating_dataframe("SpecialOffer", "SpecialOffer/")
df_Store=creating_dataframe("Store", "Store/")
df_Currency=creating_dataframe("Currency", "Currency/")
df_SpecialOfferProduct=creating_dataframe("SpecialOfferProduct", "SpecialOfferProduct/")